# Creating a Json file containing information of all variables

This notebook creates a json dictionary file 'full_var_dict.json' containing the original variable name, a descriptive variable name, as well as a brief description of each of the values inside each variable. The goal of this file is to ease the EDA and to produce graphs easier to read and understand with informative titles and descriptions of the categories of each variable.

In [33]:
import pandas as pd
import json

The excel file is the original data codebook, obtained from the INE website (e.g., Instituto Nacional de Estadistica), the owner of the data.

Tables 1 to 4 explain in detail the meaning of each of the values inside each of the categorical values in the data as well as how missing data is coded for numerical and categorical variables

In [34]:
# Specify the Excel file path
excel_file_path = '../data/raw/disreg_enceursalud20_a.xlsx'

# List of sheet names
sheet_names = ['Tablas1', 'Tablas2', 'Tablas3', 'Tablas4']

This code iterates trough tables 1 trough 4 and creates one single JSNON 'var_keys_dict' file with the description of each of the categorical values in the data as well as how missing data is coded

In [35]:
#pip install openpyxl
# Initialize variables
tables = {}

# Iterate through sheets
for sheet_name in sheet_names:
    # Read the Excel sheet and drop the specified column
    df = pd.read_excel(excel_file_path, sheet_name).drop('Unnamed: 2', axis=1, errors='ignore')

    # Initialize variables
    current_table = None
    table_data = {}

    # Iterate through rows
    for index, row in df.iterrows():
        # Check if the current row is the start of a new table
        if isinstance(row['Unnamed: 0'], str) and not row['Unnamed: 0'].isdigit() and row['Unnamed: 0'] != 'Código ':
            # Save the previous table data
            if current_table is not None:
                tables[current_table] = table_data

            # Set the current table name and initialize an empty dictionary for table data
            current_table, table_data = row['Unnamed: 0'], {}

        # Check if the row contains data (skip rows with NaN values)
        if pd.notna(row['Unnamed: 0']) and (isinstance(row['Unnamed: 0'], int) or (isinstance(row['Unnamed: 0'], str) and row['Unnamed: 0'].isdigit())):
            # Append the row data to the table_data dictionary
            table_data[row['Unnamed: 0']] = row['Unnamed: 1']

    # Save the last table data for the current sheet
    if current_table is not None:
        tables[current_table] = table_data

# Save the combined data to a JSON file
json_file_path = '../data/json_files/var_keys_dict.json'
with open(json_file_path, 'w', encoding='utf-8') as json_file:
    json.dump(tables, json_file, indent=2, ensure_ascii=False)

print(f'Successfully converted {excel_file_path} to {json_file_path}')

Successfully converted ../data/raw/disreg_enceursalud20_a.xlsx to ../data/json_files/var_keys_dict.json


This code creates a separate JSON file 'var_dict.json' containing the original variable name in the data with a brief description of the variable. The variable names in the data are short and uninformative. Hence the need of having a dictionary that explains what the variable name means

In [36]:
df = pd.read_excel(excel_file_path, 'Diseño', usecols=['Variable','Descripción','Diccionario de la variable']).set_index('Variable')

#renames the column with a shorter name
df.dropna(subset='Diccionario de la variable', inplace=True)
df.rename(columns={'Diccionario de la variable':'diccionario'}, inplace=True)

#converts the variable dictionary into json format
json_file_path = '../data/json_files/var_dict.json'
with open(json_file_path, 'w', encoding='utf-8') as json_file:
    df.to_json(json_file, orient='index', lines=False, indent=2,force_ascii=False, default_handler=str)

This code opens the two formerly created JSON files and combines them into a single file 'full_var_dict' containing: variable name, description, and description of each of the categorical values and/or missing values for each variable

In [37]:
# Load JSON strings into dictionaries
path1 = '../data/json_files/var_keys_dict.json'
path2 = '../data/json_files/var_dict.json'

with open(path1, 'r') as json_file: #loads dictionary variables
    var_keys_dict = json.load(json_file)

with open(path2, 'r') as json_file: #loads json value keys of variables
    var_dict = json.load(json_file)

#THESE TWO CONDITIONALS MODIFY THE VARIABLE KEYS TO BINARY FOR THE TRANSFORMED BINARY VARIABLES IN
# THE DATA CLEANING NOTEBOOK
if 'TSINO' in var_keys_dict:
    tsino_data = var_keys_dict['TSINO']
    
    # Create a new key '0' with the modified values
    tsino_data['0'] = f"{tsino_data['2']}/{tsino_data['8']} - {tsino_data['9']}"
    
    # Remove the unnecessary keys
    tsino_data.pop('2')
    tsino_data.pop('8')
    tsino_data.pop('9')

if 'T1SINO' in var_keys_dict:
    tsino_data = var_keys_dict['T1SINO']
    
    # Create a new key '0' with the modified values
    tsino_data['0'] = f"{tsino_data['2']}"
    # Remove the unnecessary keys
    tsino_data.pop('2')

# Iterate through the first JSON (var_dict) and replace values
for key, value in var_dict.items():
    if "diccionario" in value and value["diccionario"] in var_keys_dict:
        var_dict[key]["diccionario"] = var_keys_dict[value["diccionario"]]

# Storing the resulting JSON (as full_var_dict)
json_file_path = '../data/json_files/full_var_dict.json'
with open(json_file_path, 'w', encoding='utf-8') as json_file:
    json.dump(var_dict, json_file, indent=2, ensure_ascii=False)

# Final details for preparing JSON final file for deployment

Since some variables used in the model were previously transformed into binary format, variable 'P87_2a' and 'P87_13a' will be replaced with 'P87_2a_nuevo' and 'P87_13a_nuevo' so that the JSON file containing all variable info contains the binary variables

In [38]:
json_file_path = '../data/json_files/full_var_dict.json'
with open(json_file_path, 'r') as json_file_full: #loads full json file 
    full_var_dict = json.load(json_file_full)

if 'P87_2a' in full_var_dict:
    full_var_dict['P87_2a_nuevo'] = full_var_dict.pop('P87_2a')

if 'P87_13a' in full_var_dict:
    full_var_dict['P87_13a_nuevo'] = full_var_dict.pop('P87_13a')

#Updating the description of the variables
full_var_dict['P87_13a_nuevo']['Descripción'] ='Medicinas para el estómago y/o las alteraciones digestivas consumidas o recetadas'
full_var_dict['P87_2a_nuevo']['Descripción'] = 'Medicinas para el dolor consumidas o recetadas'

# Write the modified JSON data back to the file
with open('../data/json_files/full_var_dict.json', 'w', encoding='utf-8') as file:
    json.dump(full_var_dict, file, indent=2, ensure_ascii=False)

# Creating an english version of the JSON for deployment

List of the variables used by the model:

In [39]:
#selecting deployment vars
depl_vars=["G21","G24a", "G25a_5","G25a_6","G25a_7","G25a_8","G25a_14","G25a_18","G25a_24","G25a_25","G25a_27","K37","K38","K38a","L45","N48","N58_2","P87_2a_nuevo","P87_13a_nuevo"]

This code will select the model variables from the full dictionary file with them containing all their information in Spanish:

In [40]:
selected_dict = {key: full_var_dict[key] for key in depl_vars}
print(selected_dict)

{'G21': {'diccionario': {'1': 'Muy bueno', '2': 'Bueno', '3': 'Regular', '4': 'Malo', '5': 'Muy malo'}, 'Descripción': 'Estado de salud percibido en los últimos 12 meses'}, 'G24a': {'diccionario': {'1': 'Muy bueno', '2': 'Bueno', '3': 'Regular', '4': 'Malo', '5': 'Muy malo', '8': 'No sabe', '9': 'No contesta'}, 'Descripción': 'Estado de salud bucodental'}, 'G25a_5': {'diccionario': {'1': 'Sí', '0': 'No/No sabe - No contesta'}, 'Descripción': 'Ha padecido alguna vez: Varices en las piernas'}, 'G25a_6': {'diccionario': {'1': 'Sí', '0': 'No/No sabe - No contesta'}, 'Descripción': 'Ha padecido alguna vez: Artrosis (excluyendo artritis)'}, 'G25a_7': {'diccionario': {'1': 'Sí', '0': 'No/No sabe - No contesta'}, 'Descripción': 'Ha padecido alguna vez: Dolor de espalda crónico (cervical)'}, 'G25a_8': {'diccionario': {'1': 'Sí', '0': 'No/No sabe - No contesta'}, 'Descripción': 'Ha padecido alguna vez: Dolor de espalda crónico (lumbar)'}, 'G25a_14': {'diccionario': {'1': 'Sí', '0': 'No/No sabe -

Using chat GPT, we translated the previously created dictionary into english:

In [41]:
selected_dict_eng={
  "G21": {
    "dictionary": {
      "1": "Very good",
      "2": "Good",
      "3": "Fair",
      "4": "Poor",
      "5": "Very poor"
    },
    "Description": "Perceived health status in the last 12 months"
  },
  "G24a": {
    "dictionary": {
      "1": "Very good",
      "2": "Good",
      "3": "Fair",
      "4": "Poor",
      "5": "Very poor",
      "8": "Don't know",
      "9": "No answer"
    },
    "Description": "Oral health status"
  },
  "G25a_5": {
    "dictionary": {
      "1": "Yes",
      "0": "No/Don't know - No answer"
    },
    "Description": "Have you ever suffered from: Varicose veins in the legs"
  },
  "G25a_6": {
    "dictionary": {
      "1": "Yes",
      "0": "No/Don't know - No answer"
    },
    "Description": "Have you ever suffered from: Osteoarthritis (excluding arthritis)"
  },
  "G25a_7": {
    "dictionary": {
      "1": "Yes",
      "0": "No/Don't know - No answer"
    },
    "Description": "Have you ever suffered from: Chronic back pain (cervical)"
  },
  "G25a_8": {
    "dictionary": {
      "1": "Yes",
      "0": "No/Don't know - No answer"
    },
    "Description": "Have you ever suffered from: Chronic back pain (lumbar)"
  },
  "G25a_14": {
    "dictionary": {
      "1": "Yes",
      "0": "No/Don't know - No answer"
    },
    "Description": "Have you ever suffered from: Urinary incontinence or urine control problems"
  },
  "G25a_18": {
    "dictionary": {
      "1": "Yes",
      "0": "No/Don't know - No answer"
    },
    "Description": "Have you ever suffered from: Chronic constipation"
  },
  "G25a_24": {
    "dictionary": {
      "1": "Yes",
      "0": "No/Don't know - No answer"
    },
    "Description": "Have you ever suffered from: Migraine or frequent headaches"
  },
  "G25a_25": {
    "dictionary": {
      "1": "Yes",
      "0": "No/Don't know - No answer"
    },
    "Description": "Have you ever suffered from: Hemorrhoids"
  },
  "G25a_27": {
    "dictionary": {
      "1": "Yes",
      "0": "No/Don't know - No answer"
    },
    "Description": "Have you ever suffered from: Osteoporosis"
  },
  "K37": {
    "dictionary": {
      "1": "No difficulty at all",
      "2": "Some difficulty",
      "3": "A lot of difficulty",
      "4": "I can't do it at all",
      "5": "Not applicable (never tried or needed to do it)",
      "8": "Don't know",
      "9": "No answer"
    },
    "Description": "Any difficulty walking 500 meters without assistance?"
  },
  "K38": {
    "dictionary": {
      "1": "No difficulty at all",
      "2": "Some difficulty",
      "3": "A lot of difficulty",
      "4": "I can't do it at all",
      "5": "Not applicable (never tried or needed to do it)",
      "8": "Don't know",
      "9": "No answer"
    },
    "Description": "Any difficulty climbing up or down 12 stairs?"
  },
  "K38a": {
    "dictionary": {
      "1": "No difficulty at all",
      "2": "Some difficulty",
      "3": "A lot of difficulty",
      "4": "I can't do it at all",
      "5": "Not applicable (never tried or needed to do it)",
      "8": "Don't know",
      "9": "No answer"
    },
    "Description": "Any difficulty concentrating?"
  },
  "L45": {
    "dictionary": {
      "1": "None",
      "2": "Very mild",
      "3": "Mild",
      "4": "Moderate",
      "5": "Severe",
      "6": "Extreme",
      "8": "Don't know",
      "9": "No answer"
    },
    "Description": "Degree of pain experienced during the last 4 weeks"
  },
  "N48": {
    "dictionary": {
      "1": "In the last 4 weeks",
      "2": "Between 4 weeks and 12 months",
      "3": "12 months ago or more",
      "4": "Never"
    },
    "Description": "When was the last visit to general or family doctor?"
  },
  "N58_2": {
    "dictionary": {
      "1": "Yes",
      "0": "No/Don't know - No answer"
    },
    "Description": "Did you visit during the last 12 months to: Psychologist, psychotherapist, or psychiatrist?"
  },
  "P87_2a_nuevo": {
    "dictionary": {
      "1": "Yes",
      "0": "No/Don't know - No answer"
    },
    "Description": "Do you take prescription or non-prescription pain medicines?"
  },
  "P87_13a_nuevo": {
    "dictionary": {
      "1": "Yes",
      "0": "No/Don't know - No answer"
    },
    "Description": "Do you take prescription or non-prescription stomach and/or digestive disorder medicines?"
  }
  }

Storing the new english dictionary into JSON format:

In [42]:
# Convert the dictionary to JSON
json_file_path = '../data/json_files/english_json.json'
with open(json_file_path, 'w', encoding='utf-8') as json_file:
    json.dump(selected_dict_eng, json_file, indent=2, ensure_ascii=False)